[Reference](https://medium.com/@kylejones_47003/bayesian-time-series-forecasting-using-orbin-ml-and-prophet-in-python-181077b3b388)

In [1]:
pip install orbit-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.9/446.9 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 906.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8

In [12]:
data

,0,1,2,3,4,5,6
0,1749,1,1749.042,96.7,-1.0,-1,1
1,1749,2,1749.123,104.3,-1.0,-1,1
2,1749,3,1749.204,116.7,-1.0,-1,1
3,1749,4,1749.288,92.8,-1.0,-1,1
4,1749,5,1749.371,141.7,-1.0,-1,1
...,...,...,...,...,...,...,...
3308,2024,9,2024.706,141.1,19.5,1047,1
3309,2024,10,2024.791,166.4,23.9,893,0
3310,2024,11,2024.873,152.5,20.9,681,0
3311,2024,12,2024.958,154.5,25.6,572,0


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from orbit.utils.dataset import load_iclaims
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data
from orbit.diagnostics.metrics import smape
from prophet import Prophet

# Define RMSE function
def rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted) ** 2))

# Load sample data
data = pd.read_csv("https://raw.githubusercontent.com/Branden-Kang/Time-Series-Analysis/refs/heads/master/Data/SN_m_tot_V2.0.csv",delimiter=';', names = ['Year', 'Month', 'Decimal year', 'SNvalue' , 'SNerror', 'Nb observations'], header=None)
data["Month"] = pd.to_datetime(data["Month"])

# Create separate dataframes for Orbit and Prophet with correct column names
orbit_data = data.copy()
prophet_data = data.copy()

# Rename columns for Orbit
orbit_data = orbit_data.rename(columns={
    "Month": "date",
    "Sunspot": "response"  # Changed from "value" to "response"
})

# Rename columns for Prophet
prophet_data = prophet_data.rename(columns={
    "Month": "ds",
    "Sunspot": "y"
})

# Split the data
train_size = len(data) - 48

# Orbit train/test split
orbit_train = orbit_data.iloc[:train_size]
orbit_test = orbit_data.iloc[train_size:]

# Prophet train/test split
prophet_train = prophet_data.iloc[:train_size]
prophet_test = prophet_data.iloc[train_size:]

# --- Orbit Model ---
model_orbit = DLT(
    response_col="response",
    date_col="date",
    seasonality=12,
)

# Fit Orbit model
model_orbit.fit(df=orbit_train)
predictions_orbit = model_orbit.predict(df=orbit_data)

# --- Prophet Model ---
# Initialize and fit Prophet model
model_prophet = Prophet(yearly_seasonality=True)
model_prophet.fit(prophet_train)

# Make predictions with Prophet
future = prophet_data[['ds']]
predictions_prophet = model_prophet.predict(future)

# Create subplot for both models
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Plot Orbit results
ax1.plot(orbit_data['date'], orbit_data['response'], label='Actual', alpha=0.5)
ax1.plot(predictions_orbit['date'], predictions_orbit['prediction'], label='Predicted', color='red')
ax1.fill_between(predictions_orbit['date'],
                predictions_orbit['prediction_5'],
                predictions_orbit['prediction_95'],
                color='red',
                alpha=0.1)
ax1.axvline(x=orbit_train['date'].iloc[-1], color='black', linestyle='--', label='Train/Test Split')
ax1.legend()
ax1.set_title('Orbit Model Forecast')

# Plot Prophet results
ax2.plot(prophet_data['ds'], prophet_data['y'], label='Actual', alpha=0.5)
ax2.plot(predictions_prophet['ds'], predictions_prophet['yhat'], label='Predicted', color='green')
ax2.fill_between(predictions_prophet['ds'],
                predictions_prophet['yhat_lower'],
                predictions_prophet['yhat_upper'],
                color='green',
                alpha=0.1)
ax2.axvline(x=prophet_train['ds'].iloc[-1], color='black', linestyle='--', label='Train/Test Split')
ax2.legend()
ax2.set_title('Prophet Model Forecast')

plt.tight_layout()
plt.savefig("timeseries_comparison.png")
plt.show()

# Calculate metrics for test set - Orbit
test_predictions_orbit = predictions_orbit.iloc[train_size:]
test_actual = orbit_test['response']

print("\nOrbit Test Set Metrics:")
print("SMAPE:", smape(test_actual, test_predictions_orbit['prediction']))
print("RMSE:", rmse(test_actual, test_predictions_orbit['prediction']))

# Calculate metrics for test set - Prophet
test_predictions_prophet = predictions_prophet.iloc[train_size:]
print("\nProphet Test Set Metrics:")
print("SMAPE:", smape(test_actual, test_predictions_prophet['yhat']))
print("RMSE:", rmse(test_actual, test_predictions_prophet['yhat']))

# Show prediction intervals for both models
print("\nOrbit Prediction Intervals:")
print(predictions_orbit[["prediction", "prediction_5", "prediction_95"]].head())

print("\nProphet Prediction Intervals:")
print(predictions_prophet[["yhat", "yhat_lower", "yhat_upper"]].head())